In [1]:
import numpy as np
import pandas as pd

In [22]:
ccf_df = pd.read_csv("../data/credit_card_fraud.csv")

In [23]:
ccf_df = pd.concat([ccf_df]*10).reset_index()

In [24]:
ccf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   index                                    80000 non-null  int64  
 1   Transaction Date and Time                80000 non-null  object 
 2   Transaction Amount                       80000 non-null  float64
 3   Cardholder Name                          80000 non-null  object 
 4   Card Number (Hashed or Encrypted)        80000 non-null  object 
 5   Merchant Name                            80000 non-null  object 
 6   Merchant Category Code (MCC)             80000 non-null  int64  
 7   Transaction Location (City or ZIP Code)  80000 non-null  object 
 8   Transaction Currency                     80000 non-null  object 
 9   Card Type                                80000 non-null  object 
 10  Card Expiration Date                     80000

In [25]:
ccf_df.head()

,index,Transaction Date and Time,Transaction Amount,Cardholder Name,Card Number (Hashed or Encrypted),Merchant Name,Merchant Category Code (MCC),Transaction Location (City or ZIP Code),Transaction Currency,Card Type,...,CVV Code (Hashed or Encrypted),Transaction Response Code,Transaction ID,Fraud Flag or Label,Previous Transactions,Transaction Source,IP Address,Device Information,User Account Information,Transaction Notes
0,0,2022-09-24 13:54:27,285.88,Shray Soman,daca51bffe0fc4eaaa7c430917f94b75fd893712492254...,"Rajagopalan, Ghose and Kant",3590,Khammam,INR,MasterCard,...,01299ac65733b5a3d774265fbfe8396b8611e5e3321855...,5,b7f69cbc-a03d-41f8-adca-75920b0242c3,1,NaN,Online,18.106.240.6,Tablet,NaN,Consequatur corporis minima ad vero deserunt n...
1,1,2020-07-24 11:20:13,1777.32,Lakshit Bakshi,d5366dd9be3a0266c0252baceaa6332210ed4dbd0ef252...,Sule PLC,7277,Vasai-Virar,EUR,American Express,...,ad8fa913d25b3970c6efa8ca504da8ba670ce2a9cf012d...,12,ad53cc8e-8412-422e-8cad-4176daac8387,1,NaN,Online,212.48.185.128,Mobile,jloyal,Dolore repellendus odio deleniti. Eaque quibus...
2,2,2023-03-18 01:05:36,3939.01,Riya Bobal,4737a6384aa1c3c31f7768b86633d6a8401dd6ab4a8ff1...,Badal PLC,9297,Nangloi Jat,USD,Visa,...,eb3be230bbd2844b1f5d8f2e4fab9ffba8ab22cfeeb69c...,5,f09cecd6-0d74-4551-8089-a31d64e6b9c5,1,3 or more,In-Person,17.190.112.46,Mobile,NaN,Unde beatae perspiciatis sapiente. Voluptates ...
3,3,2021-01-07 21:53:04,376.44,Mohanlal Balakrishnan,4721806eed8f2663bb597ff13e79a294de318358fbb54b...,Konda-Sodhi,5686,Ramagundam,USD,Visa,...,891d46993a36d78392247c642138cede01d9841daab1d9...,5,b208ae0b-4c4f-428f-b6b9-5360b288b947,0,2,In-Person,153.136.24.104,Tablet,NaN,Cupiditate repellendus necessitatibus quo occa...
4,4,2021-12-16 06:22:24,1687.33,Mannat Rout,9deacc3a9efd6e382826d400620aca5f23ed94327578e5...,Dua Ltd,2940,Adoni,INR,MasterCard,...,48a1a756f2d83f1dc57bbf14052b70a6f40d0fceed6662...,5,6b4e4e43-5b73-4906-9973-299a1b2a5e71,1,2,Online,196.153.28.131,Desktop,NaN,Molestias assumenda consectetur itaque veritat...


In [26]:
ccf_df["Card Expiration Date"][0].split("/")

['04', '29']

In [27]:
%%timeit
ccf_df["Card Expiration Date"].apply(lambda x: x.split("/"))

8.79 ms ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%%timeit
ccf_df["Card Expiration Date"].str.split("/")

10.4 ms ± 93 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
[value.split('/') for value in ccf_df["Card Expiration Date"].tolist()]

5.52 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
